# Experiment Analysis

This notebook shows you how to create and run a generic experiment using the ExperimentAnalysis class.

The experiment can be 1D or 2D.

In [ ]:
import os
import numpy as np
from qibo.gates import M
from qibo.models import Circuit
import qililab as ql
from qililab.experiment.portfolio import Exp, ExperimentAnalysis
from qililab.platform import Platform
from qililab.typings import ExperimentOptions, ExperimentSettings, Parameter
from qililab.utils import Loop, Wait

os.environ["RUNCARDS"] = "./runcards"
os.environ["DATA"] = "./data"
platform = ql.build_platform(name="soprano_master_galadriel")
platform.connect()
platform.turn_on_instruments()
platform.initial_setup()

## 1D Experiment

A 1D experiment is an experiment with one or more loops, but all running in parallel. It can be created in two different ways, either by specifying the loop in the variable "experiment_loop" in the ExperimentAnalysis constructor, or by setting the loops within the experiment options "loops" variable, which is a list of loops. We show here both options for showcasing purposes. 

In [ ]:
# setup 1D experiment setting the loop through the experiment loop option
qubit = 1
circuit = Circuit(qubit + 1)
circuit.add(ql.Drag(q=qubit, theta=np.pi, phase=0))
circuit.add(Wait(q=qubit, t=0))
circuit.add(M(qubit))

wait_loop_values = np.arange(100.0, 70_000, 1000)
# Define loop over wait duration
loop = Loop(alias="2", parameter=Parameter.GATE_PARAMETER, values=wait_loop_values)

# Define experiment options
experiment_options = ExperimentOptions(
    name="experimentAnalysis",
    loops=None,
    settings=ExperimentSettings(repetition_duration=1000, hardware_average=1000),
)
        
experiment = ql.experiment.portfolio.ExperimentAnalysis(
    platform=platform, circuits=[circuit], options=experiment_options,
    experiment_loop=loop
)

In [ ]:
# setup 1D experiment setting the loop through the experiment options
qubit = 1
circuit = Circuit(qubit + 1)
circuit.add(ql.Drag(q=qubit, theta=np.pi, phase=0))
circuit.add(Wait(q=qubit, t=0))
circuit.add(M(qubit))

wait_loop_values = np.arange(100.0, 70_000, 1000)
# Define loop over wait duration
loop_0 = Loop(alias="2", parameter=Parameter.GATE_PARAMETER, values=wait_loop_values)
loop_1 = Loop(alias="4", parameter=Parameter.GATE_PARAMETER, values=wait_loop_values)

# Define experiment options
experiment_options = ExperimentOptions(
    name="experimentAnalysis",
    loops=[loop_0, loop_1],
    settings=ExperimentSettings(repetition_duration=1000, hardware_average=1000),
)
        
experiment = ql.experiment.portfolio.ExperimentAnalysis(
    platform=platform, circuits=[circuit], options=experiment_options,
    experiment_loop=None
)
experiment.build_execution()

In [ ]:
# run experiment
results = experiment.run()

In [ ]:
# fit the data
parameters = experiment.fit()

In [ ]:
# plot data with fitted parameters
experiment.plot()

## 2D Experiment

A 2D experiment is an experiment with nested loops. It can be created only by setting the two loops in the experiment options "loops" variable, which is a list of loops. 

In [ ]:
# setup 2D experiment setting the loop through the experiment options
qubit = 1
circuit = Circuit(qubit + 1)
circuit.add(ql.Drag(q=qubit, theta=np.pi, phase=0))
circuit.add(Wait(q=qubit, t=0))
circuit.add(M(qubit))

wait_values = np.arange(4., 200., 12)
if_freq_values = np.arange(5.584e9, 5.554e9, -1e6)
# Define loop over wait duration
inner_loop = Loop(alias="2", parameter=Parameter.GATE_PARAMETER, values=wait_loop_values)
outer_loop = Loop(alias=f"drive_line_q{qubit}_bus", parameter=Parameter.IF, values=if_freq_values,
                              channel_id=0, loop=inner_loop)

# Define experiment options
experiment_options = ExperimentOptions(
    name="experimentAnalysis",
    loops=[outer_loop],
    settings=ExperimentSettings(repetition_duration=1000, hardware_average=1000),
)
        
experiment = ql.experiment.portfolio.ExperimentAnalysis(
    platform=platform, circuits=[circuit], options=experiment_options,
    experiment_loop=None
)
experiment.build_execution()

In [ ]:
# run experiment
results = experiment.run()

In [ ]:
# fit the data
parameters = experiment.fit()

In [ ]:
# plot data with fitted parameters
experiment.plot()